Week3 Assignment1 
# Explore and cluster the neighborhoods in Toronto

## Part 1 Create the dataframe

In [6]:
import numpy as np
import pandas as pd 

#### Using pandas read_html to load the data

In [3]:
!pip install lxml

     |████████████████████████████████| 5.8MB 13.9MB/s eta 0:00:01


In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table=pd.read_html(url)
table

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

#### The data needed is in the first part of the table

In [8]:
df=table[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


#### Only process the cells that have an assigned borough

In [9]:
dataset=df[df['Borough']!= 'Not assigned'].reset_index(drop=True)
dataset

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


#### Change not assigned neighbourhood value into same as its borough

In [10]:
dataset['Neighbourhood']=dataset['Neighbourhood'].replace('Not assigned', dataset['Borough'])

In [11]:
dataset.loc[dataset['Postcode']=='M7A']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Queen's Park


#### Combine cells with same Postcode

In [12]:
# group neighborhood cell with same postal code
data=pd.DataFrame(dataset.groupby(['Postcode','Borough']).apply(lambda x:', '.join(x['Neighbourhood']))).reset_index()
data

,Postcode,Borough,0
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
type(data)

pandas.core.frame.DataFrame

##### Chagne columns name to be the same as it in picture

In [13]:
data.columns=['PostalCode','Borough','Neighborhood']
#pd.set_option('display.max_rows', None) 
data.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [14]:
data.shape

(103, 3)

---------------------------------------------------------------------------------------------------------

## Part 2 Get the latitude and longitude coordinates of each neighborhood

Try geocoder, Not working

In [15]:
!conda install -c conda-forge geocoder --yes
import geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [69]:
#geocoder.google is not working, and I use geocoder.arcgis
#g = geocoder.arcgis('M1B, Toronto, Ontario') 
#g.latlng
def get_latlong(postal_code):
    df1=[]
    for postal_code in enumerate(postal_code):
        lat_lng_coords = None
        while  lat_lng_coords == None:
            g =  geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
            lat_lng_coords = g.latlng
            borough = data[data['PostalCode']==postal_code]['Borough']
            nei = data[data['PostalCode']==postal_code]['Neighborhood']
            return lat_lng_coords, borough, nei
        df1.append([(postal_code, borough, nei,
                       lat_lng_coords[0],
                           lat_lng_coords[1])]) 
    return(df1)

In [70]:
dataTor= get_latlong(postal_code=data['PostalCode'])
dataTor

([43.811525000000074, -79.19551746399998],
 Series([], Name: Borough, dtype: object),
 Series([], Name: Neighborhood, dtype: object))

Try to use pandas read csv function, and load the csv file of latitude and longitude into dataframe

In [18]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded')

Data downloaded


In [20]:
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merge data into the dataframe

In [21]:
datanew=pd.merge(data, geo)
datanew

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


---------------------------------------------------------------------------------------------------------

## Part3  Explore and cluster the neighborhoods in Toronto

In [92]:
address = ' Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronta are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronta are 43.653963, -79.387207.


In [101]:
#Create a map of Toronto using latitude and longitude
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(datanew['Latitude'], datanew['Longitude'], datanew['Borough'], datanew['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto    

In [162]:
df3=datanew[datanew['Borough'].str.contains('Toronto')]
df3.reset_index(drop=True,inplace=True)
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


Draw the map with data that contains 'Toronto' in Borough

In [99]:
map_torontonew = folium.Map(location=[latitude,longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torontonew)  
map_torontonew

You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

#### Define Foursquare Credentials and Version

In [116]:
CLIENT_ID = 'UCSG5DB04AHVG5H4T3VVBBPAP5R1MH54IBU41G3J5GEYSGNJ' 
CLIENT_SECRET = 'TOLB3BTQEASYNO1TCFD25FHECKOERJVZBCCRQEY0OELS4ATW'
VERSION = '20191101' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: UCSG5DB04AHVG5H4T3VVBBPAP5R1MH54IBU41G3J5GEYSGNJ
CLIENT_SECRET:TOLB3BTQEASYNO1TCFD25FHECKOERJVZBCCRQEY0OELS4ATW


In [107]:
import requests

#### Examine the url to see neighborhood venues

In [163]:
df3[df3['PostalCode']=='M5W']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846


In [164]:
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    43.646435,
    -79.374846,
    radius,
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=UCSG5DB04AHVG5H4T3VVBBPAP5R1MH54IBU41G3J5GEYSGNJ&client_secret=TOLB3BTQEASYNO1TCFD25FHECKOERJVZBCCRQEY0OELS4ATW&v=20191101&ll=43.646435,-79.374846&radius=500&limit=100'

In [128]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc919279ba3e5002cb1905b'},
 'response': {'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': 43.6509350045, 'lng': -79.36863880889362},
   'sw': {'lat': 43.641934995499994, 'lng': -79.38105319110639}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05ef964a520d8f620e3',
       'name': 'Hockey Hall Of Fame (Hockey Hall of Fame)',
       'location': {'address': '30 Yonge St.',
        'crossStreet': 'at Brookfield Pl.',
        'lat': 43.646974451431056,
        'lng': -79.37732323942681,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.646974451431056,
          'lng': -79.377

In [156]:
v=results["response"]['groups'][0]['items']
v[0]['venue']['categories'][0]['name']#['location']['lat']
#['categories'][0]['name']

'Museum'

### define a function to repeat process to call all postal code in dataframe

In [180]:
def getNearbyVenues(postal_code, latitudes, longitudes):
    
    venues_list=[]
    for postal_code, lat, lng in zip(postal_code, latitudes, longitudes):
       # print(postal_code)
            
        # create the API request URL
        LIMIT =100
        radius=500
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postal_code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return nearby_venues

In [181]:
Toronto_venues = getNearbyVenues(postal_code=df3['PostalCode'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude'])

In [183]:
Toronto_venues

,PostalCode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,M4E,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
...,...,...,...,...,...,...,...
1673,M7Y,43.662744,-79.321558,The Ashbridge Estate,43.664691,-79.321805,Garden
1674,M7Y,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1675,M7Y,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1676,M7Y,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop


### To check how many venues were returned for each neighborhood

In [184]:
Toronto_venues.groupby('PostalCode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4E,6,6,6,6,6,6
M4K,42,42,42,42,42,42
M4L,19,19,19,19,19,19
M4M,37,37,37,37,37,37
M4N,4,4,4,4,4,4
M4P,7,7,7,7,7,7
M4R,21,21,21,21,21,21
M4S,32,32,32,32,32,32
M4T,2,2,2,2,2,2


### To find out how many unique categories can be curated from all the returned venues

In [185]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


### Clean the data by PostalCode and it's venue categories

In [186]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['PostalCode'] = Toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
Toronto_onehot.shape

(1678, 230)

### Group rows by postal code and taking the mean of the frequency of occurrence of each categories

In [189]:
Toronto_grouped = Toronto_onehot.groupby('PostalCode').mean().reset_index()
Toronto_grouped

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.023810,0.000000,...,0.000000,0.00000,0.023810,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,M4L,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.054054,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027
4,M4N,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619
7,M4S,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.03125,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.066667,0.000000,...,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000


In [190]:
Toronto_grouped.shape

(38, 230)

In [191]:
num_top_venues = 5

for hood in Toronto_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
                  venue  freq
0                   Pub  0.17
1     Health Food Store  0.17
2          Neighborhood  0.17
3                  Park  0.17
4  Other Great Outdoors  0.17


----M4K----
                    venue  freq
0        Greek Restaurant  0.21
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.07
4  Furniture / Home Store  0.05


----M4L----
                venue  freq
0                Park  0.11
1           Pet Store  0.05
2  Italian Restaurant  0.05
3   Fish & Chips Shop  0.05
4        Burger Joint  0.05


----M4M----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2  American Restaurant  0.05
3               Bakery  0.05
4   Italian Restaurant  0.05


----M4N----
                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2                 Swim School  0.25
3                    Bus Line  0.25
4           Afghan Restaurant  0.00




### Put it into a pandas dataframe

In [192]:
#define a function to sort venues in descending order 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### create the new dataframe and display the top 10 venues for each postal code

In [194]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcode_venues_sorted = pd.DataFrame(columns=columns)
postalcode_venues_sorted['PostalCode'] = Toronto_grouped['PostalCode']

for ind in np.arange(Toronto_grouped.shape[0]):   
    postalcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

postalcode_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Park,Other Great Outdoors,Neighborhood,Health Food Store,Trail,Pub,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner
1,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Spa,Bookstore,Brewery,Bubble Tea Shop
2,M4L,Park,Board Shop,Pub,Brewery,Sandwich Place,Burger Joint,Burrito Place,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop
3,M4M,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Diner,Park,Neighborhood,Middle Eastern Restaurant
4,M4N,Bus Line,Park,Construction & Landscaping,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster Neighborhoods

Run K-Means to cluster the borough into 7 clusters

In [196]:
from sklearn.cluster import KMeans

In [210]:
    # set number of clusters
    kclusters = 7

    Toronto_grouped_clustering = Toronto_grouped.drop('PostalCode', 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

    # check cluster labels generated for each row in the dataframe
    kmeans.labels_[0:10] 

array([5, 1, 1, 1, 0, 6, 1, 1, 4, 1], dtype=int32)

Create a new dataframe that includes the cluster and the top 10 venues for each neighborhood.

In [211]:
# add clustering labels
postalcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(postalcode_venues_sorted.set_index('PostalCode'), on='PostalCode')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,5,Park,Other Great Outdoors,Neighborhood,Health Food Store,Trail,Pub,Discount Store,Dessert Shop,Dim Sum Restaurant,Diner
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Spa,Bookstore,Brewery,Bubble Tea Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Park,Board Shop,Pub,Brewery,Sandwich Place,Burger Joint,Burrito Place,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Diner,Park,Neighborhood,Middle Eastern Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Construction & Landscaping,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Visualize the resulting cluster

In [215]:
import matplotlib.cm as cm
import matplotlib.colors as colors


In [216]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters